In [3]:
import pandas as pd
from understat import Understat
import asyncio
import json
import nest_asyncio
import aiohttp
from matplotlib import pyplot as plt
import requests

In [57]:
# Функция, выполняющая асинхронный запрос к базе UnderStat
nest_asyncio.apply()

async def get_league(league:object, year_season:int) -> pd.DataFrame:
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        leagues = (await understat.get_league_table(league_name=league, season = year_season))
            
    return pd.DataFrame(leagues[1:], columns=leagues[0])

async def get_players(team:object, year_season:int) -> pd.DataFrame:
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        players = await understat.get_team_players(team_name=team, season=year_season)
    return pd.DataFrame(players)

async def get_team_results(team_name:str, year_season:int) -> pd.DataFrame:
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        player_data = await understat.get_team_results(team_name=team_name, season=year_season)
    return pd.DataFrame(player_data)

async def get_match_players (match_id:int):
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        match_players = await understat.get_match_players(match_id=match_id)
    return pd.DataFrame(match_players)

async def get_match_xPTS ():
    async with aiohttp.ClientSession() as session:
        understat = Understat(session)
        match_stats = await understat.get_teams(league_name='EPL', season=2023, title = 'Tottenham')
    return pd.DataFrame(match_stats[0]['history'])

In [58]:
def get_best_performers(league:object, season:int) -> pd.DataFrame:

    columns = ['id', 'player_name', 'games', 'time', 'goals', 'xG', 'assists', 'xA',
       'shots', 'key_passes', 'yellow_cards', 'red_cards', 'position',
       'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup']
    df = pd.DataFrame(columns  = columns)

    loop = asyncio.get_event_loop()
    league_table = loop.run_until_complete(get_league(league=league, year_season=season))
    for team in league_table[5:]['Team']:
        all_players = loop.run_until_complete(get_players(team=team, year_season=season))
        df = pd.concat([df, all_players], ignore_index=True)
    df_ = df

    df_['xGChain'] = df_['xGChain'].astype(float)
    df_['time'] = df_['time'].astype(float)
    df_['games'] = df_['games'].astype(int)

    df_['xGChain90'] = df_['xGChain']/df_['time']*90

    df_ = pd.merge(df_, league_table, how= 'left', left_on='team_title', right_on='Team')

    columns_ = ['player_name', 'Team', 'position', 'games', 'time', 'xGChain', 'xGBuildup', 'xGChain90', 'xG_y', 'id']
    df_ = df_[columns_]
    df_['impact'] = df_['xGChain']/df_['xG_y']
    df_['season'] = season
    df_['mean_time_per_match'] = df_['time']/df_['games']

    best_performers_overall = df_[(df_['impact'] >= df_['impact'].quantile(0.85)) & (df_['xGChain90'] >= df_['xGChain90'].quantile(0.85))].sort_values(by = 'impact', ascending=False).reset_index(drop=True)
    return best_performers_overall

In [59]:
def search_player(player_name:str) -> json:

	url = "https://transfermarkt-db.p.rapidapi.com/v1/search/full-search"

	querystring = {"query":player_name,"search_type":"players","locale":"UK","page_number":"0"}

	headers = {
		"X-RapidAPI-Key": "---",
		"X-RapidAPI-Host": "transfermarkt-db.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	return (response.json())

In [60]:
def get_player_profile(player_id:int) -> json:

    url = "https://transfermarkt-db.p.rapidapi.com/v1/players/profile"

    querystring = {"locale":"UK","player_id":player_id}

    headers = {
        "X-RapidAPI-Key": "---",
        "X-RapidAPI-Host": "transfermarkt-db.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return (response.json())

In [61]:
df = pd.DataFrame(columns  = ['player_name', 'Team', 'position', 'games', 'time', 'xGChain',
       'xGBuildup', 'xGChain90', 'xG_y', 'id', 'impact', 'season'])

In [62]:
seasons = [2023]
leagues = ['La Liga', 'EPL', 'Bundesliga', 'Serie A', 'Ligue_1']

for season in seasons:
    for league in leagues:
        performers = get_best_performers(league, season)
        performers['league'] = league
        df = pd.concat([df, performers], ignore_index=True)

In [63]:
df_league_1 = df[df['league'] == 'Ligue_1'].reset_index(drop=True)
df_league_1

,player_name,Team,position,games,time,xGChain,xGBuildup,xGChain90,xG_y,id,impact,season,mean_time_per_match,league
0,Pierre-Emerick Aubameyang,Marseille,F S,28,2140.0,21.677333,3.976026186719537,0.911664,47.32,318,0.458101,2023,76.428571,Ligue_1
1,Florian Sotoca,Lens,D F M S,26,2079.0,16.139787,9.079641874879599,0.698692,48.02,3348,0.336106,2023,79.961538,Ligue_1
2,Akor Adams,Montpellier,F S,26,1881.0,13.063273,1.1851117042824626,0.625037,43.97,11783,0.297095,2023,72.346154,Ligue_1
3,Amine Gouiri,Rennes,F M S,25,1778.0,11.528479,4.738947289995849,0.583556,44.97,6401,0.256359,2023,71.120000,Ligue_1
4,Vitinha,Marseille,F S,18,921.0,11.574844,4.518646661192179,1.131092,47.32,11380,0.244608,2023,51.166667,Ligue_1
5,Sepe Elye Wahi,Lens,F S,22,1354.0,11.687243,1.8116889568045735,0.776848,48.02,9182,0.243383,2023,61.545455,Ligue_1
6,Eli Junior Kroupi,Lorient,F M S,24,937.0,7.165681,2.316689965315163,0.688272,29.54,11504,0.242576,2023,39.041667,Ligue_1
7,Iliman Ndiaye,Marseille,F M S,24,1495.0,11.061555,5.43021663883701,0.665913,47.32,9307,0.233761,2023,62.291667,Ligue_1
8,Ismaila Sarr,Marseille,F M S,20,1255.0,9.185030,4.213070567697287,0.658687,47.32,5675,0.194105,2023,62.750000,Ligue_1
9,Adrien Thomasson,Lens,M S,25,1117.0,8.966183,3.574163783341646,0.722432,48.02,3549,0.186718,2023,44.680000,Ligue_1


In [75]:
players_ids = []
for player in df['player_name']:
    player_data = search_player(player)
    try:
        # First try to handle 'players' as a list
        players_ids.append(player_data['data']['players'][0]['id'])
    except TypeError:
        try:
            # If that fails, try to handle 'players' as a dictionary
            players_ids.append(player_data['data']['players']['id'])
        except (TypeError, KeyError):
            # If both attempts fail, append None
            players_ids.append(None)
    except KeyError:
        # This will catch the case where 'players' or 'id' doesn't exist
        players_ids.append(None)

df['player_transermarkt_id'] = players_ids

In [81]:
df = df.dropna(subset=['player_transermarkt_id'])
df['player_transermarkt_id'] = df['player_transermarkt_id'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 0 to 73
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   player_name             73 non-null     object 
 1   Team                    73 non-null     object 
 2   position                73 non-null     object 
 3   games                   73 non-null     object 
 4   time                    73 non-null     float64
 5   xGChain                 73 non-null     float64
 6   xGBuildup               73 non-null     object 
 7   xGChain90               73 non-null     float64
 8   xG_y                    73 non-null     float64
 9   id                      73 non-null     object 
 10  impact                  73 non-null     float64
 11  season                  73 non-null     object 
 12  mean_time_per_match     73 non-null     float64
 13  league                  73 non-null     object 
 14  player_transermarkt_id  73 non-null     int6

In [82]:
def enrich_data_from_transfermarkt(row):
    profile = get_player_profile(row['player_transermarkt_id'])
    profile = profile['data']['playerProfile']
    return pd.Series({
        'dateOfBirth': profile['dateOfBirth'],
        'age': profile['age'],
        'height': profile['height'],
        'foot': profile['foot'],
        'country': profile['countrynameEN'],
        'contractExpiryDate': profile['contractExpiryDate'],
        'positionGroup': profile['positionGroup'],
        'marketValue': profile['marketValue'],
        'marketValueCurrency': profile['marketValueCurrency'],
        'marketValueNumeral': profile['marketValueNumeral'],
        'marketValueLastChange': profile['marketValueLastChange'],
    })

# Assuming df is your DataFrame
new_columns = df.apply(enrich_data_from_transfermarkt, axis=1)
df = df.join(new_columns)

In [83]:
df

,player_name,Team,position,games,time,xGChain,xGBuildup,xGChain90,xG_y,id,...,age,height,foot,country,contractExpiryDate,positionGroup,marketValue,marketValueCurrency,marketValueNumeral,marketValueLastChange
0,Isco,Real Betis,M S,24,2012.0,16.398283,6.944731794297695,0.733522,40.24,2255,...,21,"1,70",left,Portugal,"Jun 30, 2027",Sturm,15.00,€,m,"Mar 8, 2024"
1,Iago Aspas,Celta Vigo,F M S,27,2085.0,14.860691,6.333136853761971,0.641469,41.97,2290,...,36,"1,76",left,Spain,"Jun 30, 2025",Sturm,3.00,€,m,"Mar 21, 2024"
2,Sergio Arribas,Almeria,F M S,29,1818.0,12.896004,3.5045384662225842,0.638416,37.08,8862,...,22,"1,74",left,Spain,"Jun 30, 2029",Mittelfeld,8.00,€,m,"Dec 22, 2023"
3,Ayoze Pérez,Real Betis,F M S,23,1682.0,13.232255,5.109328594058752,0.708028,40.24,770,...,30,"1,78",right,Spain,"Jun 30, 2027",Sturm,7.00,€,m,"Dec 22, 2023"
4,Samu Omorodion,Alaves,F S,28,1698.0,12.722205,2.2756215017288923,0.674322,38.87,11831,...,19,"1,93",right,Spain,"Jun 30, 2028",Sturm,40.00,€,m,"Mar 21, 2024"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Eli Junior Kroupi,Lorient,F M S,24,937.0,7.165681,2.316689965315163,0.688272,29.54,11504,...,17,"1,79",right,France,"Jun 30, 2026",Mittelfeld,15.00,€,m,"Dec 18, 2023"
70,Iliman Ndiaye,Marseille,F M S,24,1495.0,11.061555,5.43021663883701,0.665913,47.32,9307,...,24,"1,80",right,Senegal,"Jun 30, 2028",Sturm,18.00,€,m,"Dec 18, 2023"
71,Ismaila Sarr,Marseille,F M S,20,1255.0,9.185030,4.213070567697287,0.658687,47.32,5675,...,26,"1,85",right,Senegal,"Jun 30, 2028",Sturm,20.00,€,m,"Dec 18, 2023"
72,Adrien Thomasson,Lens,M S,25,1117.0,8.966183,3.574163783341646,0.722432,48.02,3549,...,30,"1,75",right,France,"Jun 30, 2026",Mittelfeld,6.00,€,m,"Dec 18, 2023"


In [84]:
df.to_csv('best_performers.csv', index=False)